In [ ]:
!pip install strands-agents[mistral] python-dotenv

In [1]:
import json
import os
import sys
import dotenv

import pprint

import yaml
from pathlib import Path
from typing import Dict, List, Tuple, TypeVar
from tqdm import tqdm

# Pydantic for structured data
from pydantic import BaseModel, Field

# Strands for AI agents
from strands.agent import Agent
from strands.models.mistral import MistralModel

from collections import Counter

sys.path.append('..')

from src.utils import (
    save_json,
    read_json,
    load_file_content,
    get_job_paths,
    get_training_paths,
    sanity_check,
	chat_with_persona,
    track_api_call,  # Cost tracking from utils
    print_cost_summary,  # Cost summary from utils
    reset_cost_tracker  # Reset cost tracker from utils
)

from src.my_utils import (
    display_markdown_file,
    call_mistral,
    get_agent,
    batch_extract,
    compute_stat_for_multi_items
)

from src.models.training_info import (TrainingInfo)
from src.prompts.training_extraction_prompt import TRAINING_EXTRACTION_PROMPT

# Load API key from .env file
dotenv.load_dotenv("../env")

# Check if we're good to go
if not os.getenv("MISTRAL_API_KEY"):
    print("❌ No MISTRAL_API_KEY found!")
    print("Create an env file with your API key")
else:
    print("✅ API key found, we're ready to roll")

NameError: name 'Enum' is not defined

In [ ]:
DATA_TRAININGS_DIR = Path('../data_trainings')

In [ ]:
with open("../src/config.yaml", "r") as f:
    config = yaml.safe_load(f)

training_data_version = config["training_data_version"]
print(f"Active training data version: {training_data_version}")

# Functions definition

In [ ]:
def extract_training_info(
    path: Path,
    model: str = "mistral-small-latest"
) -> TrainingInfo:
    """Extract training info from file using Training Extraction Agent"""
    text = load_file_content(path)
    prompt = TRAINING_EXTRACTION_PROMPT + text

    extraction_agent = get_agent(model_id=model, temperature=0.0)
    result = extraction_agent.structured_output(output_model=TrainingInfo, prompt=prompt)

    if hasattr(extraction_agent, 'last_response'):
        track_api_call(extraction_agent.last_response, model)

    return result

In [ ]:
if True:
    # Get first job file
    training_paths = get_training_paths()
    if training_paths:
        print(training_paths[0])
        sample_training = extract_training_info(training_paths[0])
        print(f"Training ID: {training_paths[0].stem}")
        print(sample_training.describe())
        print("\nRaw Pydantic model:")
        print(sample_training.model_dump())
    else:
        print("No job files found!")

# Training parsing loop

In [9]:
# Compute statistic

In [8]:
# Process all trainings
print("📂 Processing Trainings...")

filename = f"extracted_trainings_{training_data_version}.json"
trainings_save_path = DATA_TRAININGS_DIR / filename

trainings_data = batch_extract(
    get_training_paths(),
    extract_training_info,
    trainings_save_path,
    show_cost_every=50  # Show cost update every 50 items
)

# Convert to TrainingInfo objects
trainings_info = {
    training_id: TrainingInfo.model_validate_json(data)
    for training_id, data in trainings_data.items()
}

print(f"✅ Extracted {len(trainings_info)} trainings")
print("\n" + "="*50)

📂 Processing Trainings...
Processing 497 files (0 already cached)


 10%|█         | 50/497 [01:03<08:37,  1.16s/it]


💰 Cost update after 50 new items:
💰 Cost Summary:
  Total API calls: 0
  Total tokens: 0
  Estimated cost: $0.0000



 20%|██        | 100/497 [02:19<11:30,  1.74s/it]


💰 Cost update after 100 new items:
💰 Cost Summary:
  Total API calls: 0
  Total tokens: 0
  Estimated cost: $0.0000



...


💰 Cost update after 400 new items:
💰 Cost Summary:
  Total API calls: 0
  Total tokens: 0
  Estimated cost: $0.0000



 91%|█████████ | 450/497 [10:54<01:25,  1.82s/it]


💰 Cost update after 450 new items:
💰 Cost Summary:
  Total API calls: 0
  Total tokens: 0
  Estimated cost: $0.0000



100%|██████████| 497/497 [11:58<00:00,  1.45s/it]


✅ Processed 497 new items
💰 Cost Summary:
  Total API calls: 0
  Total tokens: 0
  Estimated cost: $0.0000
✅ Extracted 497 trainings



In [10]:
def compute_trainings_data_stastistics(trainings_data):
    # Extract infos from the dictionary
    level_acquired_types = []

    for training_data in trainings_data.values():
        data = json.loads(training_data)
        level_acquired_types.append(data['level_acquired'])

    type_counts = Counter(level_acquired_types)
    print("\n" + "="*50)
    print("\nlevel_acquired_types Counts:")
    for type_name, count in type_counts.most_common():
        print(f"{type_name}: {count}")
    print(f"\nTotal count: {sum(type_counts.values())}")

In [11]:
filename = f"extracted_trainings_{training_data_version}.json"
trainings_save_path = DATA_TRAININGS_DIR / filename
trainings_data = read_json(trainings_save_path)

In [12]:
compute_trainings_data_stastistics(trainings_data)



level_acquired_types Counts:
Basic: 166
Advanced: 166
Intermediate: 165

Total count: 497
